# Can you SEE MY SCREEN?

<span style="color: red;"> 
How do we connect LLMs with Tools?
</span>


In [ ]:
! pip install openai==1.59.6

In [1]:
# if you don't have an API key as an environment variable you can set it here

import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
import os

client = OpenAI()

In [3]:
def llm_response(prompt: str):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

llm_response("How do we connect LLMs with Tools? Output should be a single sentence.")

"To connect Large Language Models (LLMs) with tools, you can use an interface that bridges the LLM's outputs with the APIs or functionalities of the tools, enabling seamless communication and interaction based on the model's generated responses."

We could ask the model to:

1. Write Python code and then give the model the ability to run that code somewhere
2. Create the Python function that executes the task, then somehow give the model the ability to run that function autonomously

Option number 2 is what we call FUNCTION CALLING!

# Function Calling from Scratch

1. Write a Python function
2. Make that function "available" to the LLM model of choice (in this case will be gpt-4o/gpt-4o-mini)
3. Test if the model can execute that function properly
   1. Prepare the payload for the function (the arguments for it...)
   2. Write the function call
   3. We run the function call ourselves
   4. We inspect the results

In [4]:
def create_folder(folder_path: str):
    os.makedirs(folder_path, exist_ok=True)
    return f"Folder created at {folder_path}"

create_folder("./pancakes-are-the-best")

'Folder created at ./pancakes-are-the-best'

In [5]:
!ls -d ./* | grep pancakes

./pancakes-are-the-best


In [6]:
def write_file(file_path: str, content: str):
    with open(file_path, "w") as file:
        file.write(content)
    return f"File created at {file_path}"

write_file("./pancakes-are-the-best/pancakes-are-the-best.txt", "Pancakes are the best!")

'File created at ./pancakes-are-the-best/pancakes-are-the-best.txt'

In [7]:
!cat pancakes-are-the-best/pancakes-are-the-best.txt

Pancakes are the best!

# LLM + Python Functions

The most silly way possible first!

In [8]:
prompt_with_function_information = """
You are a personal assistant with desktop capabilities. 
Users will ask you to perform tasks and you will execute those tasks by calling one or more of the following functions:

- create_folder(folder_path: str)
- write_file(file_path: str, content: str)

For example if the user asks: 

'Create a folder called lucas-teaches-function-callling'

Your output should be code like this:
'create_folder("./lucas-teaches-function-callling")'

'Create a folder called pancakes-are-the-best and write a file called pancakes-are-the-best.txt with the content "Pancakes are the best!"'

Your output should be code like this:

['create_folder("./pancakes-are-the-best")', 'write_file("./pancakes-are-the-best/pancakes-are-the-best.txt", "Pancakes are the best!")']

Your OUTPUT should ALWAYS BE python code only.

Here is the user prompt:
"""


user_prompt = "Create a file named pancakes-are-better-than-waffles.md with a one paragraph summary for why pancakes are better, \
make sure to address EG and BS students from my course who dared to say waffles are better."

In [9]:
prompt = prompt_with_function_information + user_prompt
output_with_python_code_and_ticks = llm_response(prompt)
output_with_python_code_and_ticks

'```python\nwrite_file("./pancakes-are-better-than-waffles.md", "Pancakes are undeniably better than waffles, and to my dear EG and BS students who dared to argue otherwise, let me clarify. The fluffy texture of pancakes, combined with their ability to absorb syrup, creates a harmonious blend of flavors, making them the perfect breakfast choice. Unlike waffles, which can be overly crispy and rigid, pancakes offer a comforting, soft experience that melts in your mouth. So let\'s embrace the pancake revolution and celebrate the culinary delight they bring to our breakfast tables!")\n```'

Let's clean up this output a little bit:

In [10]:
output_cleaned = output_with_python_code_and_ticks.replace("```python\n", "").replace("```", "")

output_cleaned

'write_file("./pancakes-are-better-than-waffles.md", "Pancakes are undeniably better than waffles, and to my dear EG and BS students who dared to argue otherwise, let me clarify. The fluffy texture of pancakes, combined with their ability to absorb syrup, creates a harmonious blend of flavors, making them the perfect breakfast choice. Unlike waffles, which can be overly crispy and rigid, pancakes offer a comforting, soft experience that melts in your mouth. So let\'s embrace the pancake revolution and celebrate the culinary delight they bring to our breakfast tables!")\n'

Let's find a way to execute this code!

In [11]:
exec(output_cleaned)

In [12]:
!ls | grep pancakes

pancakes-are-better-than-waffles.md
pancakes-are-the-best


We could improve this a little bit by writing up a couple of functions that do a slightly better job at cleaning up the output generated by the LLM,
and then executing the resulting function call:

In [13]:
import re

def parse_llm_function_calls(llm_output: str):
    """
    Naive parser that looks for function calls in the model output (e.g. 'create_folder("test")')
    and returns a list of dicts, each containing 'function_name' and 'args'.
    """
    calls = []
    
    # Regex looks for function_name( ... ) 
    # capturing everything until the next matching parenthesis
    pattern = r"(\w+)\(([^)]*)\)"
    matches = re.findall(pattern, llm_output)
    
    for match in matches:
        function_name = match[0]
        raw_args = match[1].split(',')
        
        # Clean up quotes/spaces
        parsed_args = [arg.strip().strip('"').strip("'") for arg in raw_args]
        calls.append({
            "function_name": function_name,
            "args": parsed_args
        })
    
    return calls

def execute_llm_function_calls(calls):
    """
    Executes each call by mapping the function_name to an actual Python function.
    If a function is unknown, handle gracefully.
    """
    for call in calls:
        fn_name = call["function_name"]
        args = call["args"]
        
        if fn_name == "create_folder":
            create_folder(*args)
        elif fn_name == "write_file":
            write_file(*args)
        else:
            print(f"[Warning] Unknown function: {fn_name}")

# Example usage
# Suppose the model returned the following text (as a single string):
# 'create_folder("./some_folder") write_file("./some_folder/readme.txt","Some content")'

model_generated_code = """
create_folder("./some_folder")
write_file("./some_folder/readme.txt", "Some content")
"""

# 1) Parse
calls = parse_llm_function_calls(model_generated_code)

# 2) Execute
execute_llm_function_calls(calls)

In [14]:
!ls some_folder/

readme.txt


We connected LLM + FUNCTION via some simple PROMPT magic!

In [15]:
!ls -d */ | grep some

some_folder/
